## 1. Scene-specific_system

이전에 구현 하였던 Scene-specific method 에서는 씬에따라 Detector(Head)를 유저가 선택해야한다는 불편한점을 개선하기 위하여 위의 시스템이 구현됨.

### 명령어

* python Cdetect.py  --weights runs/train/yolov5s_new/weights/best.pt && cd.. && cd yolov5_detect && python detect.py --weights runs/train/yolov5s_new/weights/best.pt

### 또는 명령어 아이콘 클릭 

click_cmd 파일 아이콘 클릭으로 실행

```
Scene-specific_system
    --- yolov5_infer
        --- fil1.1
        --- fil1.2
        --- fil1.3
        --- click_cmd (here)
        --- fil1.n
        
    --- yolov5_ detect
```

폴더는 크게 두가지로 나뉜다.

1. yolov5_infer
* yolov5_infer는 현재 들어오는 씬에서 어떠한 오브젝트 사이즈들이 분포 하는지 파악 하는 과정을 가진다.

2. yolov5_detect
* yolov5_detect는 yolov5_infer에서 파악된 과정의 결과 값을 가져와 어떠한 Detector가 현재 씬에 가장 적절한지 선택 후 추론과정을 가진다.

### 1.1 yolov5_infer

각 오브젝트 사이즈를 지정한 Detector(head) 별 들어오는 씬의 오브젝트 전체 갯수를 Sum하고 이후 이러한 전체 갯수를 통해 각 Detector의 결과값을 가진다.

File: /Scene-specific_system/yolov5_infer/Cdetect.py
아래의 파일은 Cdetect.py의 line 52 ~ 53과 line 116 ~ 256을 나타낸다.

1. line 52 ~ 53은 각 Detector의 개별 수행을 반복적으로 진행해주기 위해 checkfile이 필요하며 최종 이러한 과정일 끝난 후 결과 값을 가지기 위해 sumfile이 필요하다.


2. line 116 ~ 256은 4번의 inference 과정을 가지게 되고 각 추론 과정 마다 위에서 정한 Detector가 순차적으로 입력된다. # Run inference  _ 1 ~ 4 

In [ ]:
# line 52 ~ 53

checkfile = "C:/Users/dlwlsgh/testworks_final/Scene-specific_system/yolov5_infer/CheckFile.txt"
sumfile = "C:/Users/dlwlsgh/testworks_final/Scene-specific_system/yolov5_infer/sum.txt"

In [5]:
# line 116 ~ 256

    # Run inference  _ 1 
    model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup
    seen, windows, dt = 0, [], [0.0, 0.0, 0.0]
    detection_result = 0
    data_sum = 0
    for path, im, im0s, vid_cap, s in dataset:
        with open(checkfile, "w") as file:
            file.write("1")
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred = model(im, augment=augment, visualize=visualize) 
        t3 = time_sync()
        dt[1] += t3 - t2   

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # results of detect
        modi = pred[0] 
        modi1 = modi.size()
        detection_result = (modi1[0]) # 각 이미지별 디텍션 결과 갯수
        #print(f"1. {detection_result}")
        data_sum += detection_result

    with open(sumfile, 'a') as file:  
        data1 = f"{data_sum}\n"              
        file.write(str(data1))
        data_sum = 0
            
    # Run inference  _ 2
    for path, im, im0s, vid_cap, s in dataset:
        with open(checkfile, "w") as file:
            file.write("2")
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred = model(im, augment=augment, visualize=visualize) 
        t3 = time_sync()
        dt[1] += t3 - t2   

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # results of detect
        modi = pred[0] 
        modi1 = modi.size()
        detection_result = (modi1[0]) # 각 이미지별 디텍션 결과 갯수
        #print(f"1. {detection_result}")
        data_sum += detection_result

    with open(sumfile, 'a') as file:  
        data1 = f"{data_sum}\n"              
        file.write(str(data1))
        data_sum = 0
    
    # Run inference  _ 3
    for path, im, im0s, vid_cap, s in dataset:
        with open(checkfile, "w") as file:
            file.write("3")
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred = model(im, augment=augment, visualize=visualize) 
        t3 = time_sync()
        dt[1] += t3 - t2   

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # results of detect
        modi = pred[0] 
        modi1 = modi.size()
        detection_result = (modi1[0]) # 각 이미지별 디텍션 결과 갯수
        #print(f"1. {detection_result}")
        data_sum += detection_result

    with open(sumfile, 'a') as file:  
        data1 = f"{data_sum}\n"              
        file.write(str(data1))
        data_sum = 0

    # Run inference  _ 4
    for path, im, im0s, vid_cap, s in dataset:
        with open(checkfile, "w") as file:
            file.write("4")
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred = model(im, augment=augment, visualize=visualize) 
        t3 = time_sync()
        dt[1] += t3 - t2   

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # results of detect
        modi = pred[0] 
        modi1 = modi.size()
        detection_result = (modi1[0]) # 각 이미지별 디텍션 결과 갯수
        #print(f"1. {detection_result}")
        data_sum += detection_result

    with open(sumfile, 'a') as file:  
        data1 = f"{data_sum}\n"              
        file.write(str(data1))
        data_sum = 0

        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f'{i}: '
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
                   

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(f'{txt_path}.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        #print(f"# Add bbox to image :{c}\n")

                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        annotator.box_label(xyxy, label, color=colors(c, True))
                    if save_crop:
                        save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

            # Stream results
            im0 = annotator.result()
            if view_img:
                if platform.system() == 'Linux' and p not in windows:
                    windows.append(p)
                    cv2.namedWindow(str(p), cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)  # allow window resize (Linux)
                    cv2.resizeWindow(str(p), im0.shape[1], im0.shape[0])
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                        save_path = str(Path(save_path).with_suffix('.mp4'))  # force *.mp4 suffix on results videos
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer[i].write(im0)

        # Print time (inference-only)
        LOGGER.info(f'{s}Done. ({t3 - t2:.3f}s)')



최종 추론과정이 완료되면 File: /Scene-specific_system/yolov5_infer/sum.txt 파일에 각 Detector별 결과값이 기록된다.

예시)
1. 명령어
* python Cdetect.py  --weights runs/train/yolov5s_new/weights/best.pt

2. txt 결과 값
* 2623
* 2806
* 2916
* 2942

### 1.2 yolov5_detect

yolov5_infer에서 추론된 txt 결과 값을 통해 현재 씬에서 무엇이 가장 적절한지 판단함.
이러한 과정을 수행하기 위해 위의 txt 결과값을 퍼센트 단위로 변경함.

File: /Scene-specific_system/yolov5_detect/yolo.py
아래의 파일은 Cdetect.py의 line 43 ~ 51을 나타낸다.

- txt 파일을 yolo에서 불러온 이후 percent 단위로 변경.


In [ ]:
# File: /Scene-specific_system/yolov5_detect/yolo.py (line 43 ~ 51)

inferPath = "C:/Users/dlwlsgh/testworks_final/Scene-specific_system/yolov5_infer/sum.txt"

with open(inferPath, "r") as file:
    strings = file.readlines()
    temp = list(map(int, strings))
    

# temp [0], [1], [2] ,[3] = head1, head2, head3, head4(all)
h1, h2, h3, h4 = percentCompare(temp[0], temp[1], temp[2], temp[3])

File: /Scene-specific_system/yolov5_detect/utils/function.py
아래의 파일은function.py의 line 54 ~ 59를 나타낸다.

이후 txt 파일의 결과 값을 4개의 헤드를 전부 사용하는 Detector를 기준으로 퍼센트 변환과정을 거친다.

In [ ]:
# File: /Scene-specific_system/yolov5_detect/utils/function.py (line 54 ~ 59)

def percentCompare(h1, h2, h3, h4):
     h1 = round(h1 / h4 * 100)
     h2 = round(h2 / h4 * 100)
     h3 = round(h3 / h4 * 100)
     h4 = round(h4 / h4 * 100)
     return h1, h2, h3, h4

이후 결과값은 서로 비교하는 과정을 가진다. 
전체 헤드를 사용하는 Detector를 기준으로 5% 이내일 경우 또는 3% 등 아래부터 순차적으로 선택된다.

File: /Scene-specific_system/yolov5_detect/yolo.py
아래의 파일은 Cdetect.py의 line 53 ~ 76을 나타낸다.


In [ ]:
hSum = [h1, h2, h3, h4]

print(hSum)

nano_ = h4 - h1
small_ = h4 - h2
medium_ = h4 - h3
compa_medium = h3 - h2

if nano_ <= 3: 
    start, end, head = 22, 30, 1
    print("Nano")
elif small_ <= 3:
    start, end, head = 25, 30, 2
    print("Small")
elif compa_medium <= 1:
    start, end, head = 25, 30, 2
    print("Small")     
elif medium_ <= 5:
    start, end, head = 28, 30, 3 
    print("Medium")     
else: 
    start, end, head = 0, 0, 1
    print("Large")

## 2. 시작전 확인 사항

경로의 시작점은 항상 "Scene-specific_system\yolov5_infer"의 경로에서 전체 명령어가 수행되어야 함
- 전체 명령어 예제
- - testworks_final\Scene-specific_system\yolov5_infer> python Cdetect.py  --weights runs/train/yolov5s_new/weights/best.pt && cd.. && cd yolov5_detect && python detect.py --weights runs/train/yolov5s_new/weights/best.pt

수행이 정상적으로 진행되어 있지 않은 경우 yolov5_infer폴더 내 sum.txt 내에서 모든 값을 remove 해줘야 함. 

## 1.1 클릭 커맨드

click_cmd 파일 아이콘 클릭으로 실행

```
Scene-specific_system
    --- yolov5_infer
        --- click_cmd


    --- yolov5_ detect
```